In [1]:
from torchvision import datasets

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Lambda
from torch import optim

# 1. Import Data

In [2]:
training_data = datasets.CIFAR10(root='cifar10', 
                                 train=True, 
                                 download=True, 
                                 transform=ToTensor())

test_data = datasets.CIFAR10(root='cifar10', 
                             train=False, 
                             download=True, 
                             transform=ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [3]:
print(f'Training data: {training_data}')
print(f'Test data: {test_data}')

Training data: Dataset CIFAR10
    Number of datapoints: 50000
    Root location: cifar10
    Split: Train
    StandardTransform
Transform: ToTensor()
Test data: Dataset CIFAR10
    Number of datapoints: 10000
    Root location: cifar10
    Split: Test
    StandardTransform
Transform: ToTensor()


In [4]:
print(f'Image dimensions: {training_data[0][0].shape}')

Image dimensions: torch.Size([3, 32, 32])


In [5]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

# 2. Neural Network

* [ ] He initialization
* [x] ReLU activation function **(hidden layer)**
* [x] Softmax activation function **(output layer)**
* [x] Cross Entropy Loss
* [x] Adam optimizer
* [ ] Dropout 
* [ ] Batch Normalization

In [6]:
# get GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [7]:
class NeuralNetwork(nn.Module):
    
    def __init__(self, dims):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(dims[0], dims[1]),
            nn.ReLU(),
            nn.Linear(dims[1], dims[2]),
            nn.ReLU(),
            nn.Linear(dims[2], dims[3])
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        
        return logits

In [8]:
image_size = training_data[0][0].shape[0] * training_data[0][0].shape[1] * training_data[0][0].shape[2]
num_labels = 10

dims = [image_size, 100, 100, num_labels]
model = NeuralNetwork(dims=dims).to(device)

print(model)

NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=10, bias=True)
  )
)


# 3. Fit and Predict

In [9]:
def fit(dataloader, model, loss_fn, optimizer, print_loss=False):
    """ Fit neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
        optimizer:
            optimizer function, as pytorch object.
        print_loss:
            print loss on every batch, as boolean (default False)
    """
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        # compute prediction
        pred = model(X)
        
        # compute loss
        loss = loss_fn(pred, Y)

        # reset the gradients
        optimizer.zero_grad()
        
        # backpropagate
        loss.backward()
        
        # update parameters
        optimizer.step()

        if print_loss and batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [10]:
def predict(dataloader, model, loss_fn):
    """ Predict with neural network.
    
    Args:
        dataloader:
            pytorch DataLoader object.
        model:
            neural network, as pytorch object.
        loss_fn:
            loss function, as pytorch object.
    """
    size = len(dataloader.dataset)
    
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for X, Y in dataloader:
            pred = model(X)
            
            test_loss += loss_fn(pred, Y).item()
            
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"average loss: {test_loss:>8f} \naccuracy: {(100*correct):>0.1f}%\n")

# 4. Predict on test data

In [11]:
batch_size = 64
learning_rate = 1e-3
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    fit(train_dataloader, model, loss_fn, optimizer, print_loss=True)
    
    print('\nTrain:\n-------')
    predict(train_dataloader, model, loss_fn)
    
    print('\nTest:\n-------')
    predict(test_dataloader, model, loss_fn)
    
    print(f"-------------------------------")

Epoch 1
-------------------------------
loss: 2.328216 [    0/50000]
loss: 2.015908 [ 6400/50000]
loss: 1.956435 [12800/50000]
loss: 1.924407 [19200/50000]
loss: 1.662471 [25600/50000]
loss: 1.798967 [32000/50000]
loss: 1.826696 [38400/50000]
loss: 1.792024 [44800/50000]

Train:
-------
average loss: 0.027232 
accuracy: 36.9%


Test:
-------
average loss: 0.027373 
accuracy: 36.5%

-------------------------------
Epoch 2
-------------------------------
loss: 1.802806 [    0/50000]
loss: 1.636673 [ 6400/50000]
loss: 1.658983 [12800/50000]
loss: 1.773434 [19200/50000]
loss: 1.668064 [25600/50000]
loss: 1.729813 [32000/50000]
loss: 1.532233 [38400/50000]
loss: 1.734095 [44800/50000]

Train:
-------
average loss: 0.025559 
accuracy: 40.7%


Test:
-------
average loss: 0.025846 
accuracy: 40.4%

-------------------------------
Epoch 3
-------------------------------
loss: 1.850076 [    0/50000]
loss: 1.727802 [ 6400/50000]
loss: 1.850490 [12800/50000]
loss: 1.587694 [19200/50000]
loss: 1.36